# First let's generate some images using the generative model

In [10]:
!pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

from IPython.display import Image, display

In [16]:
class Generator(nn.Module):
    def __init__(self, g_input_dim, g_output_dim):
        super(Generator, self).__init__()       
        self.fc1 = nn.Linear(g_input_dim, 256)
        self.fc2 = nn.Linear(self.fc1.out_features, self.fc1.out_features*2)
        self.fc3 = nn.Linear(self.fc2.out_features, self.fc2.out_features*2)
        self.fc4 = nn.Linear(self.fc3.out_features, g_output_dim)
    # forward method
    def forward(self, x): 
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = F.leaky_relu(self.fc3(x), 0.2)
        return torch.tanh(self.fc4(x))

In [13]:
import random

z_dim = 100
mnist_dim = 784
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

G = Generator(g_input_dim=z_dim, g_output_dim=mnist_dim).to(device)
G.load_state_dict(torch.load('./state_dicts/generator.pth'))

def gen_new_mnist():
    id = random.randint(0, 10**5)
    filename = f'./samples/sample_{id}.png'
    with torch.no_grad():
        test_z = Variable(torch.randn(1, z_dim).to(device))
        generated = G(test_z)
        save_image(generated.view(generated.size(0), 1, 28, 28), filename)
    return filename

In [14]:
import os
files = os.listdir("./samples")
file_count = len(files)
total = 1000
remaining = total - file_count

In [17]:

images = []
for _ in range(remaining):
    images.append(Image(filename=gen_new_mnist()))
images

[<IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.core.display.Image object>,
 <IPython.co

# Now it's time to rank them

In [19]:
type(images[0])

IPython.core.display.Image

In [21]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 4.9 MB 22.1 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 76.6 MB/s eta 0:00:01
     |████████████████████████████████| 163 kB 89.5 MB/s eta 0:00:01


In [22]:
from transformers import AutoFeatureExtractor, AutoModelForImageClassification

extractor = AutoFeatureExtractor.from_pretrained("farleyknight-org-username/vit-base-mnist")

model = AutoModelForImageClassification.from_pretrained("farleyknight-org-username/vit-base-mnist")

Downloading:   0%|          | 0.00/228 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/985 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [24]:
from transformers import pipeline

pipe = pipeline("image-classification", model="farleyknight-org-username/vit-base-mnist", device=0)
pipe

In [28]:
type(images[0])

IPython.core.display.Image

In [30]:
filenames = [img.filename for img in images]
filenames

['./samples/sample_80751.png',
 './samples/sample_55341.png',
 './samples/sample_45682.png',
 './samples/sample_69078.png',
 './samples/sample_80940.png',
 './samples/sample_38313.png',
 './samples/sample_35096.png',
 './samples/sample_2683.png',
 './samples/sample_38451.png',
 './samples/sample_82721.png',
 './samples/sample_14742.png',
 './samples/sample_7518.png',
 './samples/sample_22892.png',
 './samples/sample_71894.png',
 './samples/sample_11698.png',
 './samples/sample_44245.png',
 './samples/sample_22517.png',
 './samples/sample_24413.png',
 './samples/sample_99906.png',
 './samples/sample_20832.png',
 './samples/sample_66698.png',
 './samples/sample_99250.png',
 './samples/sample_13588.png',
 './samples/sample_26650.png',
 './samples/sample_31100.png',
 './samples/sample_32483.png',
 './samples/sample_39872.png',
 './samples/sample_17238.png',
 './samples/sample_48104.png',
 './samples/sample_8503.png',
 './samples/sample_3102.png',
 './samples/sample_5933.png',
 './samples/s

In [32]:
result = pipe(images=filenames)

In [37]:
full_report = [{'filename': filenames[i], 'image': Image(filename=filenames[i]), 'predictions': result[i]} for i in range(len(filenames))]
full_report

[{'filename': './samples/sample_80751.png',
  'image': <IPython.core.display.Image object>,
  'predictions': [{'score': 0.9590373635292053, 'label': '3'},
   {'score': 0.024427596479654312, 'label': '5'},
   {'score': 0.00536513514816761, 'label': '2'},
   {'score': 0.003626341000199318, 'label': '8'},
   {'score': 0.0024983661714941263, 'label': '1'}]},
 {'filename': './samples/sample_55341.png',
  'image': <IPython.core.display.Image object>,
  'predictions': [{'score': 0.9925865530967712, 'label': '0'},
   {'score': 0.0016710144700482488, 'label': '5'},
   {'score': 0.001100938650779426, 'label': '7'},
   {'score': 0.0009133262792602181, 'label': '6'},
   {'score': 0.0008818657952360809, 'label': '3'}]},
 {'filename': './samples/sample_45682.png',
  'image': <IPython.core.display.Image object>,
  'predictions': [{'score': 0.995171844959259, 'label': '2'},
   {'score': 0.001370440935716033, 'label': '6'},
   {'score': 0.0007748949574306607, 'label': '7'},
   {'score': 0.0006489588413

In [54]:
high_quality_images = []

for index in range(len(full_report)):
    row = full_report[index]
    first_pred = row['predictions'][0]
    first_pred_score = first_pred['score']
    first_pred_label = first_pred['label']
    row['highest_score'] = first_pred_score
    if first_pred_score >= 0.995:
        high_quality_images.append(index)
        
print('How many high quality images do we have?', len(high_quality_images))

How many high quality images do we have? 554


In [64]:
full_report.sort(key=lambda x: -x['highest_score'])

for i in range(200):
    display(full_report[i]['image'])

In [68]:
import json

json_report = json.dumps(full_report)
with open("./full_report_on_samples.json", 'w') as f:
    f.write(json_report)

# TODO

* Create a bunch of images [DONE]
* Rank the images [DONE]
* Save the results to a JSON file so they can be re-used [DONE]
* Determine the "High Quality" images [DONE]
* Copy the HQ images to the digit-recognizer-app [TODO]
* Add them to my report [TODO]
* Finally call that section done [TODO]